In [2]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import pandas as pd
import numpy as np
import plotly.express as px 
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import haystack_utils
import hook_utils
import plotting_utils
import probing_utils
from probing_utils import get_and_score_new_word_probe
from sklearn import preprocessing

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

model = HookedTransformer.from_pretrained("EleutherAI/pythia-160m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

german_data = haystack_utils.load_json_data("data/german_europarl.json")[:200]
english_data = haystack_utils.load_json_data("data/english_europarl.json")[:200]

LAYER, NEURON = 8, 2994

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


In [11]:
with open(f'data/pythia_160m/layer_8/single_neurons_df.pkl', 'rb') as f:
    one_sparse_probe_scores_df = pickle.load(f)

with open(f'data/pythia_160m/layer_8/neuron_and_2994_df.pkl', 'rb') as f:
    two_sparse_probe_scores_df = pickle.load(f)

top_one_scores = one_sparse_probe_scores_df.sort_values(by='mcc', ascending=False).head(10)
# top_one_scores_indices = top_one_scores.index
print(top_one_scores.index)

# for i in range(model.cfg.d_mlp):
#     one_scores = one_sparse_probe_scores_df.loc[i]
#     if one_scores['mcc'] > 0.62:
#         print(f'N{i}', one_scores['f1'], one_scores['mcc'])
for i in range(model.cfg.d_mlp):
    two_scores = two_sparse_probe_scores_df.loc[i]
    if two_scores['mcc'] > 0.75:
        print(f'N{i}_N2994', two_scores['f1'], two_scores['mcc'])

            f1       mcc
1426  0.859275  0.742336
149   0.795104  0.641232
2994  0.817568  0.637815
1815  0.803144  0.625381
1080  0.821160  0.624540
626   0.760399  0.622951
2288  0.789410  0.621133
2636  0.811525  0.614484
1404  0.816850  0.613347
2830  0.775984  0.608591
N1426_N2994 0.8784987612671973 0.7724234688752558
